**Experiment 1**: use linear transformation matrix to weight collocates.
See http://www.dialog-21.ru/media/3908/enikeevaevmitrofanovaoa.pdf

In [1]:
from gensim.models.word2vec import Word2Vec

In [2]:
import numpy as np

In [3]:
from sklearn.cross_validation import train_test_split

In [4]:
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

## Utility functions

In [21]:
def train_test_split_(coll_data, test_word=None, b_pos='_S', c_pos='_V'):
    B = []
    C = []
    collset = []
    target_b_vec, target_c_vec, target_coll = [], [], []
    for x in coll_data:
        b_vec, c_vec = 0, 0
        if test_word:
            if x[0] == test_word:
                try:
                    target_c_vec.append(c_model[x[1].split('_')[0]])
                    target_b_vec.append(b_model[x[0].split('_')[0] + b_pos])
                    target_coll.append(x)
                except:
                    pass
                continue
            continue
        try:
            b_vec = b_model[x[0].split('_')[0] + b_pos]
            c_vec = c_model[x[1].split('_')[0]]
        except KeyError:
            pass
        if type(b_vec) != int and type(c_vec) != int:
            collset.append(x)
            B.append(b_vec)
            C.append(c_vec)
    if test_word:
        return np.array(B), target_m_vec, collset, target_coll, np.array(C), target_c_vec

    return train_test_split(np.array(B), collset, np.array(C))

## Load models and precompute weights

In [6]:
pm_corr = {
    '_V': "INFN",
    '_S': "NOUN",
    '_A': "ADJF",
    '_ADV': "ADVB"
}

In [35]:
def most_similar_baseline(self, pos_w, neg_w, topn=5, pos='_V', use_pm=False):
    best = self.most_similar_cosmul(pos_w, neg_w, topn=topn*5)
    if use_pm:
        return list(filter(lambda x: filter_pm_pos(x[0], pos), best))[:topn]
    return list(filter(lambda x: x[0].endswith(pos), best))[:topn]

In [8]:
def filter_pm_pos(word, pos):
    parses = morph.parse(word)
    if pos in ('_A', '_S'):
        return pm_corr[pos] in [p.tag.POS for p in parses] \
            and np.all(['Apro' not in p.tag for p in parses])
    return pm_corr[pos] in [p.tag.POS for p in morph.parse(word)]

In [9]:
def most_similar_by_wv(self, vectenter, topn=5, pos='_V', use_pm=False):
    best = self.similar_by_vector(vectenter, topn=topn*5)
    if use_pm:
        return list(filter(lambda x: filter_pm_pos(x[0], pos), best))[:topn]
    return list(filter(lambda x: x[0].endswith(pos), best))[:topn]

In [10]:
def most_similar_vector(self, vectenter, topn=5, pos='_V', use_pm=False):
    self.init_sims()
    dists = np.dot(self.syn0norm, vectenter)
    if not topn:
        return dists
    best = np.argsort(dists)[::-1][:topn*5]
        # ignore (don't return) words from the input
    result = [(self.index2word[sim], float(dists[sim])) for sim in best]
    if use_pm:
        return list(filter(lambda x: filter_pm_pos(x[0], pos), result))[:topn]
    return list(filter(lambda x: x[0].endswith(pos), result))[:topn]

In [11]:
b_model = Word2Vec.load_word2vec_format('data/ruscorpora.model.bin', binary=True)

In [12]:
c_model = Word2Vec.load_word2vec_format('data/wiki-ru.vec.bin', binary=True)

In [13]:
c_model.init_sims(replace=True)

## Load train/test data

In [15]:
def load_links(lf='attr', b_pos='_S', c_pos='_A'):
    coll_ = []
    with open('data/synt_links_%s' % lf, 'r') as f:
        for line in f:
            line = line.rstrip('\r\n').split('\t')
            if pm_corr[c_pos] in [x.tag.POS for x in morph.parse(line[0])] \
            and pm_corr[b_pos] in [x.tag.POS for x in morph.parse(line[1])]:
                pair = (line[1].lower() + b_pos, line[0].lower() + c_pos)
                if not pair in coll_:
                    coll_.append(pair)
    return coll_

In [16]:
def load_syntagrus_lf(lf='_OPER1', b_pos='_S', c_pos='_V'):
    coll_lf = []
    with open('data/lf.SynTagRus.tsv', 'r') as f:
        for line in f:
            line = line.rstrip('\r\n').split('\t')
            if line[0] == lf:
                pair = (line[1].lower() + b_pos, line[2].lower() + c_pos)
                if not pair in coll_lf:
                    coll_lf.append(pair)
    return coll_lf

In [17]:
def load_fuzzy_syntagrus_lf(lf='_OPER', b_pos='_S', c_pos='_V'):
    coll_lf = []
    with open('data/lf.SynTagRus.tsv', 'r') as f:
        for line in f:
            line = line.rstrip('\r\n').split('\t')
            if lf in line[0]:
                pair = (line[1].lower() + b_pos, line[2].lower() + c_pos)
                if not pair in coll_lf:
                    coll_lf.append(pair)
    return coll_lf

In [100]:
def load_pair_to_lf(coll_lf, lf='_OPER', b_pos='_S', c_pos='_V'):
    with open('data/lf.SynTagRus.tsv', 'r') as f:
        for line in f:
            line = line.rstrip('\r\n').split('\t')
            if lf in line[0]:
                pair = (line[1].lower() + b_pos, line[2].lower() + c_pos)
                if not pair in coll_lf:
                    coll_lf[pair] = line[0]

## linear transformation

In [18]:
def lintransform(M_train, C_train):
    return np.linalg.pinv(M_train).dot(C_train).T

## Load verbal collocations

In [19]:
import re
op = re.compile(u'.*order=(?P<order>[NV]+)')
coll_ = []
with open('data/lf.verb_coll', 'r') as f:
    for line in f:
        line = line.rstrip('\r\n').split('\t')
        if 'base=действие' in line[1].split() and 'mood=субъект' in line[1].split():
            m_p = op.search(line[1])
            if m_p.group('order'):
                i = m_p.group('order').index('V')
                try:
                    collocate = line[2].split()[i]
                    pair = (line[0], morph.parse(collocate)[0].normal_form)
                    if not pair in coll_:
                        coll_.append(pair)
                except IndexError:
                    pass

In [22]:
B_train, B_test, coll_train, coll_test, C_train, C_test = train_test_split_(coll_)

## Load LF examples

Use LF arguments as base words and LF values as collocates. Score pairs from testset and find most probable LF values.

In [47]:
coll_1 = load_fuzzy_syntagrus_lf("FUNC", c_pos="_V", b_pos="_S")
coll_2 = load_fuzzy_syntagrus_lf("OPER", c_pos="_V", b_pos="_S")
coll_lf = coll_1 + coll_2
B_train, B_test, coll_train, coll_test, C_train, C_test = train_test_split_(coll_lf, b_pos="_S", c_pos="")
X = lintransform(B_train, C_train)

In [48]:
for i, test_vec in enumerate(B_test):
    trans_vec = X.dot(test_vec.T)
#     trans_vec1 = Y.dot(M_test[i].T)
    base, target_coll = coll_test[i]
    res = np.array([x for x in most_similar_by_wv(c_model, trans_vec, topn=100, pos="_V", use_pm=True)])
    j, = np.where(res[:, 0] == target_coll.split('_')[0])
    if len(j):
        print(base, target_coll, j[0])
        for w in res[:10]:
            print('\t%s\t%.4f' % (w[0], float(w[1])))
    if target_coll + '' in res:
        true_ans1 += 1

боль_S вызывать_V 64
	испытывать	0.7445
	ощущать	0.6744
	чувствовать	0.6641
	почувствовать	0.6581
	мучить	0.6448
	забеспокоиться	0.6333
	мучаться	0.6325
	возникать	0.6265
	терзать	0.6256
	обуревать	0.6242
отбор_S вести_V 8
	проходить	0.7850
	идти	0.6500
	проводиться	0.6242
	проводить	0.6033
	происходить	0.5847
	начинаться	0.5613
	осуществляться	0.5449
	производить	0.5078
	вести	0.5029
	производиться	0.5028
название_S получать_V 1
	давать	0.8182
	получать	0.6525
	иметь	0.6415
	носить	0.6403
	называть	0.5939
	давнуть	0.5715
	использовать	0.5158
	хотеть	0.5123
	существовать	0.4991
	даваться	0.4937
альпиниада_S состояться_V 3
	проводить	0.7780
	проходить	0.5927
	проводиться	0.5580
	состояться	0.5554
	проводеть	0.5351
	отыгрывать	0.5150
	устраивать	0.5134
	состяться	0.5040
	простудившиться	0.4965
	питтодрить	0.4764
контракт_S расторгать_V 3
	подписывать	0.7740
	заключать	0.7143
	подписать	0.7019
	расторгать	0.6240
	расторгнуть	0.6141
	перезаключить	0.6099
	подписасть	0.5978
	переподписывать

крюк_S делать_V 0
	делать	0.5844
	рубиться	0.5681
	слоняться	0.5637
	заматывать	0.5568
	нянчиться	0.5485
	развлекаться	0.5457
	размахиваться	0.5454
	попрошайничать	0.5449
	караулить	0.5427
	протрезветь	0.5410
задержание_S производить_V 16
	принимать	0.5789
	инициировать	0.5636
	проводить	0.5506
	вызывать	0.5465
	осуществлять	0.5414
	подвергать	0.5302
	подвергаться	0.5198
	участвовать	0.5183
	совершать	0.5089
	провестись	0.5075
матч_S состояться_V 4
	проводить	0.8656
	проводиться	0.6716
	проходить	0.6529
	провесть	0.6443
	состояться	0.6094
	проводеть	0.6051
	организовывать	0.5878
	провестись	0.5648
	устраивать	0.5545
	состяться	0.5379
ревизия_S подвергаться_V 35
	проводить	0.8026
	проходить	0.7383
	проводиться	0.6899
	состояться	0.6407
	провесть	0.6259
	организовывать	0.5979
	устраивать	0.5845
	проводеть	0.5475
	осуществлять	0.5390
	провестись	0.5382
эмиграция_S возвращаться_V 58
	делать	0.6003
	сделать	0.5376
	производить	0.5104
	совершать	0.5064
	кувеходить	0.5044
	штормить	0.4967
	ув

KeyboardInterrupt: 

## Revert relations

Use LF values as base words and LF arguments as collocates to be detected.
Join FUNC and OPER groups of LFs and learn transformation from verbs to their arguments.

In [77]:
C_train_rev, C_test_rev, coll_train_rev, coll_test_rev, B_train_rev, B_test_rev = train_test_split_(coll_lf, b_pos="_S", c_pos="")
X = lintransform(B_train_rev, C_train_rev)

In [101]:
# load mapping: (c, b) -> LF name
pair_to_lf = {}
load_pair_to_lf(pair_to_lf, "FUNC", c_pos="_V", b_pos="_S")
load_pair_to_lf(pair_to_lf, "OPER", c_pos="_V", b_pos="_S")

In [113]:
# Score top-100 LF arguments
outf = open("data/lf_top100.res", "w")
for i, test_vec in enumerate(B_test_rev):
    trans_vec = X.dot(test_vec.T)
    target_coll, base = coll_test_rev[i]
    lf = pair_to_lf[(target_coll, base)]
    if lf.startswith("_S0"): # nominalization LFs
        continue
    res = np.array([x for x in most_similar_by_wv(b_model, trans_vec, topn=500, pos="_S")])
    j, = np.where(res[:, 0] == target_coll)
    if len(j):
        for w in res[:100]:
            print('%s\t%s\t%s\t%.4f' % (lf, base, w[0], float(w[1])), file=outf)
outf.close()

----